# 第四题：实现预剪枝

## 实验内容
1. 实现使用信息增益率划分的预剪枝
2. 计算出带有预剪枝和不带预剪枝的决策树的精度，查准率，查全率和F1值(最大深度为6，使用信息增益率)，保留4位小数，四舍五入

我们会以“信息增益率(information gain ratio)”作为划分准则，构造带有预剪枝的二叉决策树  
使用的数据和第三题一样，剪枝需要使用验证集，所以数据划分策略会和第三题不同

## 1. 读取数据

In [1]:
# 导入类库
import pandas as pd
import numpy as np

In [2]:
# 导入数据
loans = pd.read_csv('data/lendingclub/lending-club-data.csv', low_memory=False)

In [3]:
# 对数据进行预处理，将safe_loans作为标记
loans['safe_loans'] = loans['bad_loans'].apply(lambda x : +1 if x==0 else -1)
del loans['bad_loans']

In [4]:
features = ['grade',              # grade of the loan
            'term',               # the term of the loan
            'home_ownership',     # home_ownership status: own, mortgage or rent
            'emp_length',         # number of years of employment
           ]
target = 'safe_loans'
loans = loans[features + [target]]

## 2. 划分训练集和测试集

In [5]:
from sklearn.utils import shuffle

In [6]:
loans = shuffle(loans, random_state = 34)

**我们使用数据的60%做训练集，20%做验证集，20%做测试集**

In [7]:
split_line1 = int(len(loans) * 0.6)
split_line2 = int(len(loans) * 0.8)
train_data = loans.iloc[: split_line1]
validation_data = loans.iloc[split_line1: split_line2]
test_data = loans.iloc[split_line2:]

## 3. 特征预处理

In [8]:
def one_hot_encoding(data, features_categorical):
    '''
    Parameter
    ----------
    data: pd.DataFrame
    
    features_categorical: list(str)
    '''
    
    # 对所有的离散特征遍历
    for cat in features_categorical:
        
        # 对这列进行one-hot编码，前缀为这个变量名
        one_encoding = pd.get_dummies(data[cat], prefix = cat)
        
        # 将生成的one-hot编码与之前的dataframe拼接起来
        data = pd.concat([data, one_encoding],axis=1)
        
        # 删除掉原始的这列离散特征
        del data[cat]
    
    return data

In [9]:
train_data = one_hot_encoding(train_data, features)

In [10]:
one_hot_features = train_data.columns.tolist()
one_hot_features.remove(target)

In [11]:
validation_tmp = one_hot_encoding(validation_data, features)
validation_data = pd.DataFrame(columns = train_data.columns)
for feature in train_data.columns:
    if feature in validation_tmp:
        validation_data[feature] = validation_tmp[feature].copy()
    else:
        validation_data[feature] = np.zeros(len(validation_tmp), dtype = 'uint8')

In [12]:
test_data_tmp = one_hot_encoding(test_data, features)
test_data = pd.DataFrame(columns = train_data.columns)
for feature in train_data.columns:
    if feature in test_data_tmp.columns:
        test_data[feature] = test_data_tmp[feature].copy()
    else:
        test_data[feature] = np.zeros(test_data_tmp.shape[0], dtype = 'uint8')

打印一下3个数据集的shape

In [13]:
print(train_data.shape, validation_data.shape, test_data.shape)

((73564, 25), (24521, 25), (24522, 25))


# 4 实现信息增益率的计算
信息熵：
$$
\mathrm{Ent}(D) = - \sum^{\vert \mathcal{Y} \vert}_{k = 1} p_k \mathrm{log}_2 p_k
$$

信息增益：
$$
\mathrm{Gain}(D, a) = \mathrm{Ent}(D) - \sum^{V}_{v=1} \frac{\vert D^v \vert}{\vert D \vert} \mathrm{Ent}(D^v)
$$

信息增益率：

$$
\mathrm{Gain\_ratio}(D, a) = \frac{\mathrm{Gain}(D, a)}{\mathrm{IV}(a)}
$$

其中

$$
\mathrm{IV}(a) = - \sum^V_{v=1} \frac{\vert D^v \vert}{\vert D \vert} \log_2 \frac{\vert D^v \vert}{\vert D \vert}
$$

计算信息熵时约定：若$p = 0$，则$p \log_2p = 0$

先实现信息熵，再实现信息增益率

In [14]:
def information_entropy(labels_in_node):
    '''
    求当前结点的信息熵
    
    Parameter
    ----------
    labels_in_node: np.ndarray, 如[-1, 1, -1, 1, 1]
    
    Returns
    ----------
    float: information entropy
    '''
    
    # 统计样本总个数
    num_of_samples = labels_in_node.shape[0]
    
    if num_of_samples == 0:
        return 0
    
    # 统计出标记为1的个数
    num_of_positive = len(labels_in_node[labels_in_node == 1])
    
    # 统计出标记为-1的个数
    num_of_negative = len(labels_in_node[labels_in_node == -1])                                                           # YOUR CODE HERE
    
    # 统计正例的概率
    prob_positive = num_of_positive*1.0 / num_of_samples
    
    # 统计负例的概率
    prob_negative = num_of_negative*1.0 / num_of_samples                                                             # YOUR CODE HERE
    
    if prob_positive == 0:
        positive_part = 0
    else:
        positive_part = prob_positive * np.log2(prob_positive)
    
    if prob_negative == 0:
        negative_part = 0
    else:
        negative_part = prob_negative * np.log2(prob_negative)
    
    return - ( positive_part + negative_part )

In [15]:
# 信息熵测试样例1
example_labels = np.array([-1, -1, 1, 1, 1])
print(information_entropy(example_labels)) # 0.97095

# 信息熵测试样例2
example_labels = np.array([-1, -1, 1, 1, 1, 1, 1])
print(information_entropy(example_labels)) # 0.86312
    
# 信息熵测试样例3
example_labels = np.array([-1, -1, -1, -1, -1, 1, 1])
print(information_entropy(example_labels)) # 0.86312

# 信息熵测试样例4
example_labels = np.array([-1] * 9 + [1] * 8)
print(information_entropy(example_labels)) # 0.99750

# 信息熵测试样例5
example_labels = np.array([1] * 8)
print(information_entropy(example_labels)) # 0

# 信息熵测试样例6
example_labels = np.array([])
print(information_entropy(example_labels)) # 0

0.9709505944546686
0.863120568566631
0.863120568566631
0.9975025463691153
-0.0
0


In [16]:
def compute_information_gain_ratios(data, features, target, annotate = False):
    '''
    计算所有特征的信息增益率并保存起来
    
    Parameter
    ----------
    data: pd.DataFrame, 带有特征和标记的数据
    
    features: list(str)，特征名组成的list
    
    target: str， 特征的名字
    
    annotate: boolean, default False，是否打印注释
    
    Returns
    ----------
    gain_ratios: dict, key: str, 特征名
                       value: float，信息增益率
    '''
    
    gain_ratios = dict()
    
    # 对所有的特征进行遍历，使用当前的划分方法对每个特征进行计算
    for feature in features:
        
        # 左子树保证所有的样本的这个特征取值为0
        left_split_target = data[data[feature] == 0][target]
        
        # 右子树保证所有的样本的这个特征取值为1
        right_split_target =  data[data[feature] == 1][target]
            
        # 计算左子树的信息熵
        left_entropy = information_entropy(left_split_target)
        
        # 计算左子树的权重
        left_weight = len(left_split_target)*1.0 / (len(left_split_target) + len(right_split_target))

        # 计算右子树的信息熵
        right_entropy = information_entropy(right_split_target)
        
        # 计算右子树的权重
        right_weight = len(right_split_target)*1.0 / (len(left_split_target) + len(right_split_target))
        
        # 计算当前结点的信息熵
        current_entropy = information_entropy(data[target])
        
        # 计算当前结点的信息增益
        gain = current_entropy-(left_weight*left_entropy+right_weight*right_entropy)                                                                                      # YOUR CODE HERE
        
        # 计算IV公式中，当前特征为0的值
        if left_weight == 0:
            left_IV = 0
        else:
            left_IV = left_weight*np.log2(left_weight)                                                                               # YOUR CODE HERE
        
        # 计算IV公式中，当前特征为1的值
        if right_weight == 0:
            right_IV = 0
        else:
            right_IV = right_weight*np.log2(right_weight)                                                                              # YOUR CODE HERE
        
        # IV 等于所有子树IV之和的相反数
        IV = - (left_IV + right_IV)
            
        # 计算使用当前特征划分的信息增益率
        # 这里为了防止IV是0，导致除法得到np.inf，在分母加了一个很小的小数
        gain_ratio = gain / (IV + np.finfo(np.longdouble).eps)
        
        # 信息增益率的存储
        gain_ratios[feature] = gain_ratio
        
        if annotate:
            print(" ", feature, gain_ratio)
            
    return gain_ratios

In [17]:
# 信息增益率测试样例1
print(compute_information_gain_ratios(train_data, one_hot_features, target)['grade_A']) # 0.02573

# 信息增益率测试样例2
print(compute_information_gain_ratios(train_data, one_hot_features, target)['grade_B']) # 0.00418

# 信息增益率测试样例3
print(compute_information_gain_ratios(train_data, one_hot_features, target)['term_ 60 months']) # 0.01971

0.025734780668006163288
0.0041754950694306059366
0.01970936271862220511


# 信息增益(拓展)
信息熵：
$$
\mathrm{Ent}(D) = - \sum^{\vert \mathcal{Y} \vert}_{k = 1} p_k \mathrm{log}_2 p_k
$$

信息增益：
$$
\mathrm{Gain}(D, a) = \mathrm{Ent}(D) - \sum^{V}_{v=1} \frac{\vert D^v \vert}{\vert D \vert} \mathrm{Ent}(D^v)
$$

计算信息熵时约定：若$p = 0$，则$p \log_2p = 0$

In [18]:
def compute_information_gains(data,features,target,annotate=False):
    information_gains = dict()
    for feature in features:
        left_split_target = data[data[feature] == 0][target]
        right_split_target = data[data[feature] == 1][target]
        left_weight = len(left_split_target)*1.0 / (len(left_split_target)+len(right_split_target))
        right_weight = len(right_split_target)*1.0 / (len(left_split_target)+len(right_split_target))
        left_entropy = information_entropy(left_split_target)
        right_entropy = information_entropy(right_split_target)
        current_entropy = information_entropy(data[target])
        information_gains[feature] = current_entropy - (right_weight*right_entropy+left_weight*left_entropy)
        if annotate:
            print(" ", feature, gain)
    return information_gains;

In [19]:
# 信息增益测试样例1
print(compute_information_gains(train_data, one_hot_features, target)['grade_A']) # 0.01759

# 信息增益测试样例3
print(compute_information_gains(train_data, one_hot_features, target)['grade_B']) # 0.00370

# 信息增益测试样例2
print(compute_information_gains(train_data, one_hot_features, target)['term_ 60 months']) # 0.01429

0.017591980178887745
0.0037049200345331545
0.014291850329400169


# 基尼指数(拓展)
数据集$D$的基尼值：

$$
\begin{aligned}
\mathrm{Gini}(D) & = \sum^{\vert \mathcal{Y} \vert}_{k=1} \sum_{k' \neq k} p_k p_{k'}\\
& = 1 - \sum^{\vert \mathcal{Y} \vert}_{k=1} p^2_k.
\end{aligned}
$$

属性$a$的基尼指数：

$$
\mathrm{Gini\_index}(D, a) = \sum^V_{v = 1} \frac{\vert D^v \vert}{\vert D \vert} \mathrm{Gini}(D^v)
$$

In [20]:
def gini(labels_in_node):
    nums_of_samples = labels_in_node.shape[0]
    if nums_of_samples == 0:
        return 0;
    nums_of_positive = len(labels_in_node[labels_in_node == 1])
    nums_of_negative = len(labels_in_node[labels_in_node == -1])
    return (nums_of_negative*nums_of_positive)*1.0/(nums_of_samples*nums_of_samples)*2;

In [21]:
def compute_gini_indices(data,features,target,annotate = False):
    gini_indices = dict()
    for feature in features:
        left_split_target = data[data[feature] == 0][target]
        right_split_target = data[data[feature] == 1][target]
        left_gini = gini(left_split_target)
        right_gini = gini(right_split_target)
        left_weight = len(left_split_target)*1.0 / (len(right_split_target)+len(left_split_target))
        right_weight = len(right_split_target)*1.0 / (len(right_split_target)+len(left_split_target))
        gini_indices[feature] = left_weight*left_gini+right_weight*right_gini
        if annotate:
            print(" ", feature, gini_index)
    return gini_indices

In [22]:
# 基尼指数测试样例1
print(compute_gini_indices(train_data, one_hot_features, target)['grade_A']) # 0.30095209649643617

# 基尼指数测试样例2
print(compute_gini_indices(train_data, one_hot_features, target)['grade_B']) # 0.30568553758823647

# 基尼指数测试样例3
print(compute_gini_indices(train_data, one_hot_features, target)['term_ 36 months']) # 0.30055418611740065

0.300952096496
0.305685537588
0.300554186117


## 5. 完成最优特征的选择 

这里我们没有实现信息增益和基尼指数的最优特征求解，感兴趣的同学可以按上一题实现

In [23]:
def best_splitting_feature(data, features, target, criterion = 'gain_ratios', annotate = False):
    '''
    给定划分方法和数据，找到最优的划分特征
    
    Parameters
    ----------
    data: pd.DataFrame, 带有特征和标记的数据
    
    features: list(str)，特征名组成的list
    
    target: str， 特征的名字
    
    criterion: str, 使用哪种指标，三种选项: 'information_gain', 'gain_ratio', 'gini'
    
    annotate: boolean, default False，是否打印注释
    
    Returns
    ----------
    best_feature: str, 最佳的划分特征的名字
    
    '''
    if criterion == 'information_gain':
        if annotate:
            print('using information gain')
        information_gains = compute_information_gains(data,features,target,annotate)
        best_feature = max(information_gains.items(),key = lambda x:x[1])[0]
        return best_feature

    elif criterion == 'gain_ratio':
        if annotate:
            print('using information gain ratio')
        
        # 得到当前所有特征的信息增益率
        gain_ratios = compute_information_gain_ratios(data, features, target, annotate)
    
        # 根据这些特征和他们的信息增益率，找到最佳的划分特征
        best_feature = max(gain_ratios.items(), key = lambda x: x[1])[0]

        return best_feature
    
    elif criterion == 'gini':
        if annotate:
            print('using gini')
        gini_indices = compute_gini_indices(data,features,target,annotate)
        best_feature = min(gini_indices.items(),key = lambda x:x[1])[0]
        return best_feature
    else:
        raise Exception("传入的criterion不合规!", criterion)

## 6. 判断结点内样本的类别是否为同一类

In [24]:
def intermediate_node_num_mistakes(labels_in_node):
    '''
    求树的结点中，样本数少的那个类的样本有多少，比如输入是[1, 1, -1, -1, 1]，返回2
    
    Parameter
    ----------
    labels_in_node: np.ndarray, pd.Series
    
    Returns
    ----------
    int：个数
    
    '''
    # 如果传入的array为空，返回0
    if len(labels_in_node) == 0:
        return 0
    
    # 统计1的个数
    num_of_one = len(labels_in_node[labels_in_node == 1])                                                              # YOUR CODE HERE
    
    # 统计-1的个数
    num_of_minus_one = len(labels_in_node[labels_in_node == -1])                                                        # YOUR CODE HERE
    
    return num_of_one if num_of_minus_one > num_of_one else num_of_minus_one

# 7. 创建叶子结点

先编写一个辅助函数majority_class，求树的结点中，样本数多的那个类是什么

In [25]:
def majority_class(labels_in_node):
    '''
        求树的结点中，样本数多的那个类是什么
    '''
    # 如果传入的array为空，返回0
    if len(labels_in_node) == 0:
        return 0
    
    # 统计1的个数
    num_of_one = len(labels_in_node[labels_in_node == 1])                                                              # YOUR CODE HERE
    
    # 统计-1的个数
    num_of_minus_one = len(labels_in_node[labels_in_node == -1])                                                        # YOUR CODE HERE
    
    return 1 if num_of_minus_one < num_of_one else -1

In [26]:
def create_leaf(target_values):
    '''
    计算出当前叶子结点的标记是什么，并且将叶子结点信息保存在一个dict中
    
    Parameter:
    ----------
    target_values: pd.Series, 当前叶子结点内样本的标记

    Returns:
    ----------
    leaf: dict，表示一个叶结点，
            leaf['splitting_features'], None，叶结点不需要划分特征
            leaf['left'], None，叶结点没有左子树
            leaf['right'], None，叶结点没有右子树
            leaf['is_leaf'], True, 是否是叶子结点
            leaf['prediction'], int, 表示该叶子结点的预测值
    '''
    # 创建叶子结点
    leaf = {'splitting_feature' : None,
            'left' : None,
            'right' : None,
            'is_leaf': True}
   
    # 数结点内-1和+1的个数
    num_ones = len(target_values[target_values == +1])
    num_minus_ones = len(target_values[target_values == -1])    

    # 叶子结点的标记使用少数服从多数的原则，为样本数多的那类的标记，保存在 leaf['prediction']
    leaf['prediction'] = majority_class(target_values)

    # 返回叶子结点
    return leaf

## 8. 递归地创建决策树
递归的创建决策树  
决策树终止的三个条件：
1. 如果结点内所有的样本的标记都相同，该结点就不需要再继续划分，直接做叶子结点即可
2. 如果结点所有的特征都已经在之前使用过了，在当前结点无剩余特征可供划分样本，该结点直接做叶子结点
3. 如果当前结点的深度已经达到了我们限制的树的最大深度，直接做叶子结点

对于预剪枝来说，实质上是增加了第四个终止条件：  
4. 如果当前结点划分后，模型的泛化能力没有提升，则不进行划分

如何判断泛化能力有没有提升？我们需要使用验证集  
就像使用训练集递归地划分数据一样，我们在递归地构造决策树时，也需要递归地将验证集进行划分，计算决策树在验证集上的精度

因为我们是递归地对决策树进行划分，所以每次计算验证集上精度是否提升时，也只是针对当前结点内的样本，因为是否对当前结点内的样本进行划分，不会影响它的兄弟结点及兄弟结点的子结点的精度

**需要完成11个部分**

In [27]:
def decision_tree_create(training_data, validation_data, features, target, criterion = 'gain_ratios', pre_pruning = False, current_depth = 0, max_depth = 10, annotate = False):
    '''
    Parameter:
    ----------
    trianing_data: pd.DataFrame, 数据

    features: iterable, 特征组成的可迭代对象

    target: str, 标记的名字

    criterion: 'str', 特征划分方法

    current_depth: int, 当前深度

    max_depth: int, 树的最大深度

    Returns:
    ----------
    dict, dict['is_leaf']          : False, 当前顶点不是叶子结点
          dict['prediction']       : None, 不是叶子结点就没有预测值
          dict['splitting_feature']: splitting_feature, 当前结点是使用哪个特征进行划分的
          dict['left']             : dict
          dict['right']            : dict
    '''
    if criterion not in ['information_gain', 'gain_ratio', 'gini']:
        raise Exception("传入的criterion不合规!", criterion)
    
    # 复制一份特征，存储起来，每使用一个特征进行划分，我们就删除一个
    remaining_features = features[:]
    
    # 取出标记值
    target_values = training_data[target]
    validation_values = validation_data[target]
    print("-" * 50)
    print("Subtree, depth = %s (%s data points)." % (current_depth, len(target_values)))

    # 终止条件1
    # 如果当前结点内所有样本同属一类，即这个结点中，各类别样本数最小的那个等于0
    # 使用前面写的intermediate_node_num_mistakes来完成这个判断
    if intermediate_node_num_mistakes(target_values) == 0:                                                                                       # YOUR CODE HERE
        print("Stopping condition 1 reached.")
        return create_leaf(target_values)   # 创建叶子结点
    
    # 终止条件2
    # 如果已经没有剩余的特征可供分割
    if len(remaining_features) == 0:                                                                                       # YOUR CODE HERE
        print("Stopping condition 2 reached.")
        return create_leaf(target_values)   # 创建叶子结点
    
    # 终止条件3
    # 如果已经到达了我们要求的最大深度
    if current_depth == max_depth:                                                                                       # YOUR CODE HERE
        print("Reached maximum depth. Stopping for now.")
        return create_leaf(target_values)   # 创建叶子结点
    
    # 找到最优划分特征
    # 使用best_splitting_feature这个函数
    splitting_feature = best_splitting_feature(training_data, features, target, criterion)                                                                      # YOUR CODE HERE
    
    # 使用我们找到的最优特征将数据划分成两份
    # 左子树的数据
    left_split = training_data[training_data[splitting_feature] == 0]
    
    # 右子树的数据
    right_split = training_data[training_data[splitting_feature] == 1]                                                                            # YOUR CODE HERE
    
    # 使用这个最优特征对验证集进行划分
    validation_left_split = validation_data[validation_data[splitting_feature] == 0]
    validation_right_split = validation_data[validation_data[splitting_feature] == 1]
    
    # 如果使用预剪枝，需要判断在验证集上的精度是否提升了
    if pre_pruning:
        # 首先计算不划分的时候的在验证集上的精度，也就是当前结点为叶子结点
        # 统计当前结点样本中，样本数多的那个类(majority class)
        true_class = majority_class(validation_values)                                                                         # YOUR CODE HERE

        # 判断验证集在不划分时的精度，分母加eps是因为，有可能在划分的时候，验证集的样本数为0
        acc_without_splitting = len(validation_values[validation_values == true_class]) / (len(validation_values) + np.finfo(np.longdouble).eps)

        # 对当前结点进行划分，统计划分后，左子树的majority class
        left_true_class = majority_class(left_split[target])

        # 对当前结点进行划分，统计右子树的majority class
        right_true_class = majority_class(right_split[target])                                                                   # YOUR CODE HERE

        # 统计验证集左子树中有多少样本是左子树的majority class
        vali_left_num_of_majority = len(validation_left_split[validation_left_split[target] == left_true_class])

        # 统计验证集右子树中有多少样本是右子树的majority class
        vali_right_num_of_majority = len(validation_right_split[validation_right_split[target] == right_true_class])                                                         # YOUR CODE HERE

        # 计算划分后的精度
        acc_with_splitting = (vali_left_num_of_majority + vali_right_num_of_majority) / (len(validation_data) + np.finfo(np.longdouble).eps)

        if annotate == True:
            print('acc before splitting: %.3f'%(acc_without_splitting))
            print('acc after splitting: %.3f'%(acc_with_splitting))

        # 如果划分后的精度小于等于划分前的精度，那就不划分，当前结点直接变成叶子结点
        # 否则继续划分，创建左右子树
        if acc_with_splitting < acc_without_splitting:
            print('Pre-Pruning')
            # 创建叶子结点
            return create_leaf(target_values)                                                                          # YOUR CODE HERE
    
    # 从剩余特征中删除掉当前这个特征
    remaining_features.remove(splitting_feature)
    
    print("Split on feature %s. (%s, %s)" % (\
                      splitting_feature, len(left_split), len(right_split)))
    
    # 如果使用当前的特征，将所有的样本都划分到一棵子树中，那么就直接将这棵子树变成叶子结点
    # 判断左子树是不是“完美”的
    if len(left_split) == len(training_data):
        print("Creating leaf node.")
        return create_leaf(left_split[target])
    
    # 判断右子树是不是“完美”的
    if len(right_split) == len(training_data):
        print("Creating right node.")
        return create_leaf(right_split[target])                                                                                # YOUR CODE HERE

    # 递归地创建左子树，需要传入验证集的左子树
    left_tree = decision_tree_create(left_split, validation_left_split, remaining_features, target, criterion, pre_pruning, current_depth + 1, max_depth, annotate)
    
    # 递归地创建右子树，需要传入验证集的右子树
    
    right_tree = decision_tree_create(right_split, validation_right_split, remaining_features, target, criterion, pre_pruning, current_depth + 1, max_depth, annotate)                                                                             # YOUR CODE HERE

    return {'is_leaf'          : False, 
            'prediction'       : None,
            'splitting_feature': splitting_feature,
            'left'             : left_tree, 
            'right'            : right_tree}

训练一棵没有预剪枝的决策树，最大深度为6

In [28]:
tree_without_pre_pruning = decision_tree_create(train_data, validation_data, one_hot_features, target, criterion = 'gain_ratio', pre_pruning = False, current_depth = 0, max_depth = 6, annotate = False)

--------------------------------------------------
Subtree, depth = 0 (73564 data points).
Split on feature grade_F. (71229, 2335)
--------------------------------------------------
Subtree, depth = 1 (71229 data points).
Split on feature grade_A. (57869, 13360)
--------------------------------------------------
Subtree, depth = 2 (57869 data points).
Split on feature grade_G. (57232, 637)
--------------------------------------------------
Subtree, depth = 3 (57232 data points).
Split on feature grade_E. (51828, 5404)
--------------------------------------------------
Subtree, depth = 4 (51828 data points).
Split on feature grade_D. (40326, 11502)
--------------------------------------------------
Subtree, depth = 5 (40326 data points).
Split on feature term_ 60 months. (34566, 5760)
--------------------------------------------------
Subtree, depth = 6 (34566 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------
Subtree, depth = 6 (57

Split on feature home_ownership_OTHER. (1717, 2)
--------------------------------------------------
Subtree, depth = 4 (1717 data points).
Split on feature emp_length_3 years. (1577, 140)
--------------------------------------------------
Subtree, depth = 5 (1577 data points).
Split on feature home_ownership_RENT. (904, 673)
--------------------------------------------------
Subtree, depth = 6 (904 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------
Subtree, depth = 6 (673 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------
Subtree, depth = 5 (140 data points).
Split on feature home_ownership_RENT. (73, 67)
--------------------------------------------------
Subtree, depth = 6 (73 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------
Subtree, depth = 6 (67 data points).
Reached maximum depth. Stopping for now.
-------------------

In [29]:
tree_without_pre_pruning_information_gain = decision_tree_create(train_data, validation_data, one_hot_features, target, criterion = 'information_gain', pre_pruning = False, current_depth = 0, max_depth = 6, annotate = False)

--------------------------------------------------
Subtree, depth = 0 (73564 data points).
Split on feature grade_A. (60204, 13360)
--------------------------------------------------
Subtree, depth = 1 (60204 data points).
Split on feature grade_B. (37768, 22436)
--------------------------------------------------
Subtree, depth = 2 (37768 data points).
Split on feature grade_C. (19878, 17890)
--------------------------------------------------
Subtree, depth = 3 (19878 data points).
Split on feature term_ 60 months. (11066, 8812)
--------------------------------------------------
Subtree, depth = 4 (11066 data points).
Split on feature home_ownership_MORTGAGE. (6987, 4079)
--------------------------------------------------
Subtree, depth = 5 (6987 data points).
Split on feature grade_F. (6650, 337)
--------------------------------------------------
Subtree, depth = 6 (6650 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------
Subtree, 

Split on feature term_ 60 months. (5830, 70)
--------------------------------------------------
Subtree, depth = 3 (5830 data points).
Split on feature emp_length_3 years. (5283, 547)
--------------------------------------------------
Subtree, depth = 4 (5283 data points).
Split on feature emp_length_1 year. (4705, 578)
--------------------------------------------------
Subtree, depth = 5 (4705 data points).
Split on feature home_ownership_OTHER. (4689, 16)
--------------------------------------------------
Subtree, depth = 6 (4689 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------
Subtree, depth = 6 (16 data points).
Stopping condition 1 reached.
--------------------------------------------------
Subtree, depth = 5 (578 data points).
Split on feature home_ownership_OTHER. (576, 2)
--------------------------------------------------
Subtree, depth = 6 (576 data points).
Reached maximum depth. Stopping for now.
----------------------

In [30]:
tree_without_pre_pruning_gini = decision_tree_create(train_data, validation_data, one_hot_features, target, criterion = 'gini', pre_pruning = False, current_depth = 0, max_depth = 6, annotate = False)

--------------------------------------------------
Subtree, depth = 0 (73564 data points).
Split on feature term_ 60 months. (58733, 14831)
--------------------------------------------------
Subtree, depth = 1 (58733 data points).
Split on feature grade_A. (45632, 13101)
--------------------------------------------------
Subtree, depth = 2 (45632 data points).
Split on feature grade_B. (25130, 20502)
--------------------------------------------------
Subtree, depth = 3 (25130 data points).
Split on feature grade_C. (11066, 14064)
--------------------------------------------------
Subtree, depth = 4 (11066 data points).
Split on feature home_ownership_MORTGAGE. (6987, 4079)
--------------------------------------------------
Subtree, depth = 5 (6987 data points).
Split on feature grade_F. (6650, 337)
--------------------------------------------------
Subtree, depth = 6 (6650 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------
Subtree,

Split on feature emp_length_2 years. (176, 19)
--------------------------------------------------
Subtree, depth = 6 (176 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------
Subtree, depth = 6 (19 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------
Subtree, depth = 3 (3185 data points).
Split on feature home_ownership_RENT. (1980, 1205)
--------------------------------------------------
Subtree, depth = 4 (1980 data points).
Split on feature emp_length_3 years. (1828, 152)
--------------------------------------------------
Subtree, depth = 5 (1828 data points).
Split on feature emp_length_10+ years. (1057, 771)
--------------------------------------------------
Subtree, depth = 6 (1057 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------
Subtree, depth = 6 (771 data points).
Reached maximum depth. Stopping for now.
------------

训练一棵有预剪枝的决策树，最大深度为6

In [31]:
tree_with_pre_pruning = decision_tree_create(train_data, validation_data, one_hot_features, target, criterion = "gain_ratio", pre_pruning = True, current_depth = 0, max_depth = 6, annotate = False)

--------------------------------------------------
Subtree, depth = 0 (73564 data points).
Split on feature grade_F. (71229, 2335)
--------------------------------------------------
Subtree, depth = 1 (71229 data points).
Split on feature grade_A. (57869, 13360)
--------------------------------------------------
Subtree, depth = 2 (57869 data points).
Split on feature grade_G. (57232, 637)
--------------------------------------------------
Subtree, depth = 3 (57232 data points).
Split on feature grade_E. (51828, 5404)
--------------------------------------------------
Subtree, depth = 4 (51828 data points).
Split on feature grade_D. (40326, 11502)
--------------------------------------------------
Subtree, depth = 5 (40326 data points).
Split on feature term_ 60 months. (34566, 5760)
--------------------------------------------------
Subtree, depth = 6 (34566 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------
Subtree, depth = 6 (57

In [32]:
tree_with_pre_pruning_information_gain = decision_tree_create(train_data, validation_data, one_hot_features, target, criterion = "information_gain", pre_pruning = True, current_depth = 0, max_depth = 6, annotate = False)

--------------------------------------------------
Subtree, depth = 0 (73564 data points).
Split on feature grade_A. (60204, 13360)
--------------------------------------------------
Subtree, depth = 1 (60204 data points).
Split on feature grade_B. (37768, 22436)
--------------------------------------------------
Subtree, depth = 2 (37768 data points).
Split on feature grade_C. (19878, 17890)
--------------------------------------------------
Subtree, depth = 3 (19878 data points).
Split on feature term_ 60 months. (11066, 8812)
--------------------------------------------------
Subtree, depth = 4 (11066 data points).
Split on feature home_ownership_MORTGAGE. (6987, 4079)
--------------------------------------------------
Subtree, depth = 5 (6987 data points).
Split on feature grade_F. (6650, 337)
--------------------------------------------------
Subtree, depth = 6 (6650 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------
Subtree, 

Split on feature emp_length_3 years. (5283, 547)
--------------------------------------------------
Subtree, depth = 4 (5283 data points).
Split on feature emp_length_1 year. (4705, 578)
--------------------------------------------------
Subtree, depth = 5 (4705 data points).
Split on feature home_ownership_OTHER. (4689, 16)
--------------------------------------------------
Subtree, depth = 6 (4689 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------
Subtree, depth = 6 (16 data points).
Stopping condition 1 reached.
--------------------------------------------------
Subtree, depth = 5 (578 data points).
Split on feature home_ownership_OTHER. (576, 2)
--------------------------------------------------
Subtree, depth = 6 (576 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------
Subtree, depth = 6 (2 data points).
Stopping condition 1 reached.
----------------------------------------

In [33]:
tree_with_pre_pruning_gini = decision_tree_create(train_data, validation_data, one_hot_features, target, criterion = "gini", pre_pruning = True, current_depth = 0, max_depth = 6, annotate = False)

--------------------------------------------------
Subtree, depth = 0 (73564 data points).
Split on feature term_ 60 months. (58733, 14831)
--------------------------------------------------
Subtree, depth = 1 (58733 data points).
Split on feature grade_A. (45632, 13101)
--------------------------------------------------
Subtree, depth = 2 (45632 data points).
Split on feature grade_B. (25130, 20502)
--------------------------------------------------
Subtree, depth = 3 (25130 data points).
Split on feature grade_C. (11066, 14064)
--------------------------------------------------
Subtree, depth = 4 (11066 data points).
Split on feature home_ownership_MORTGAGE. (6987, 4079)
--------------------------------------------------
Subtree, depth = 5 (6987 data points).
Split on feature grade_F. (6650, 337)
--------------------------------------------------
Subtree, depth = 6 (6650 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------
Subtree,

Split on feature grade_D. (1651, 1176)
--------------------------------------------------
Subtree, depth = 6 (1651 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------
Subtree, depth = 6 (1176 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------
Subtree, depth = 5 (195 data points).
Split on feature emp_length_2 years. (176, 19)
--------------------------------------------------
Subtree, depth = 6 (176 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------
Subtree, depth = 6 (19 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------
Subtree, depth = 3 (3185 data points).
Split on feature home_ownership_RENT. (1980, 1205)
--------------------------------------------------
Subtree, depth = 4 (1980 data points).
Split on feature emp_length_3 years. (1828, 152)
------------------------

# 9. 预测

In [34]:
def classify(tree, x, annotate = False):
    '''
    递归的进行预测，一次只能预测一个样本
    
    Parameters
    ----------
    tree: dict
    
    x: pd.Series，样本
    
    x: pd.DataFrame, 待预测的样本
    
    annotate, boolean, 是否显示注释
    
    Returns
    ----------
    返回预测的标记
    '''
    if tree['is_leaf']:
        if annotate:
            print ("At leaf, predicting %s" % tree['prediction'])
        return tree['prediction']
    else:
        split_feature_value = x[tree['splitting_feature']]
        if annotate:
             print ("Split on %s = %s" % (tree['splitting_feature'], split_feature_value))
        if split_feature_value == 0:
            return classify(tree['left'], x, annotate)
        else:
            return classify(tree['right'], x, annotate)

In [35]:
def predict(tree, data):
    '''
    按行遍历data，对每个样本进行预测，将值存储起来，最后返回np.ndarray
    
    Parameter
    ----------
    tree, dict, 模型
    
    data, pd.DataFrame, 数据
    
    Returns
    ----------
    predictions, np.ndarray, 模型对这些样本的预测结果
    '''
    predictions = np.zeros(len(data))
    
    # YOUR CODE HERE
    for i in range(len(data)):
        predictions[i] = classify(tree, data.iloc[i])
    
    return predictions

## 在下方计算出带有预剪枝和不带预剪枝的决策树的精度，查准率，查全率和F1值(最大深度为6，使用信息增益率)，保留4位小数，四舍五入

In [36]:
# YOUR CODE HERE
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
print("gain_ratio:")
prediction1 = predict(tree_without_pre_pruning,test_data)
print("without:")
print(accuracy_score(test_data['safe_loans'],prediction1))
print(precision_score(test_data['safe_loans'],prediction1))
print(recall_score(test_data['safe_loans'],prediction1))
print(f1_score(test_data['safe_loans'],prediction1))
prediction2 = predict(tree_with_pre_pruning,test_data)
print("with:")
print(accuracy_score(test_data['safe_loans'],prediction2))
print(precision_score(test_data['safe_loans'],prediction2))
print(recall_score(test_data['safe_loans'],prediction2))
print(f1_score(test_data['safe_loans'],prediction2))
prediction1 = predict(tree_without_pre_pruning,test_data)

print("information_gain:")
print("without:")
prediction1_information_gain = predict(tree_without_pre_pruning_information_gain,test_data)
print(accuracy_score(test_data['safe_loans'],prediction1))
print(precision_score(test_data['safe_loans'],prediction1))
print(recall_score(test_data['safe_loans'],prediction1))
print(f1_score(test_data['safe_loans'],prediction1))
prediction2_information_gain = predict(tree_with_pre_pruning_information_gain,test_data)
print("with:")
print(accuracy_score(test_data['safe_loans'],prediction2))
print(precision_score(test_data['safe_loans'],prediction2))
print(recall_score(test_data['safe_loans'],prediction2))
print(f1_score(test_data['safe_loans'],prediction2))
prediction1 = predict(tree_without_pre_pruning,test_data)

print("gini:")
print("without:")
prediction1_gini = predict(tree_without_pre_pruning_gini,test_data)
print(accuracy_score(test_data['safe_loans'],prediction1))
print(precision_score(test_data['safe_loans'],prediction1))
print(recall_score(test_data['safe_loans'],prediction1))
print(f1_score(test_data['safe_loans'],prediction1))
prediction2_gini = predict(tree_with_pre_pruning_gini,test_data)
print("with:")
print(accuracy_score(test_data['safe_loans'],prediction2))
print(precision_score(test_data['safe_loans'],prediction2))
print(recall_score(test_data['safe_loans'],prediction2))
print(f1_score(test_data['safe_loans'],prediction2))

gain_ratio:
without:
0.8088247288149417
0.8097397556890141
0.9984383658253992
0.8942429164410755
with:
0.809436424435201
0.8095024469820554
0.9998992494080903
0.8946834644249622
information_gain:
without:
0.8088247288149417
0.8097397556890141
0.9984383658253992
0.8942429164410755
with:
0.809436424435201
0.8095024469820554
0.9998992494080903
0.8946834644249622
gini:
without:
0.8088247288149417
0.8097397556890141
0.9984383658253992
0.8942429164410755
with:
0.809436424435201
0.8095024469820554
0.9998992494080903
0.8946834644249622


###### 双击此处编辑
+ gain_ratio:

模型|精度|查准率|查全率|F1
-|-|-|-|-
有预剪枝| 0.8094 | 0.8095 | 0.9999 | 0.8947
无预剪枝| 0.8088 | 0.8097 | 0.9984| 0.8942

+ information_gain:

模型|精度|查准率|查全率|F1
-|-|-|-|-
有预剪枝| 0.8094 | 0.8095 | 0.9999| 0.8947
无预剪枝| 0.0.8088 | 0.8097 | 0.9984| 0.8942

+ gini:

模型|精度|查准率|查全率|F1
-|-|-|-|-
有预剪枝| 0.8094 | 0.8095 | 0.9999 | 0.8947
无预剪枝| 0.8088 | 0.8097 | 0.9984| 0.8942

# 10. 可视化

In [37]:
from pyecharts import Tree

In [38]:
def generate_echarts_data(tree):
    
    # 当前顶点的dict
    value = dict()
    
    # 如果传入的tree已经是叶子结点了
    if tree['is_leaf'] == True:
        
        # 它的value就设置为预测的标记
        value['value'] = tree['prediction']
        
        # 它的名字就叫"label: 标记"
        value['name'] = 'label: %s'%(tree['prediction'])
        
        # 直接返回这个dict即可
        return value
    
    # 如果传入的tree不是叶子结点，名字就叫当前这个顶点的划分特征，子树是一个list
    # 分别增加左子树和右子树到children中
    value['name'] = tree['splitting_feature']
    value['children'] = [generate_echarts_data(tree['left']), generate_echarts_data(tree['right'])]
    return value

In [39]:
data1 = generate_echarts_data(tree_without_pre_pruning)

In [40]:
tree = Tree(width=800, height=800)
tree.add("",
         [data1],
         tree_collapse_interval=5,
         tree_top="15%",
         tree_right="20%",
         tree_symbol = 'rect',
         tree_symbol_size = 20,
         )
tree.render()
tree

In [41]:
data2 = generate_echarts_data(tree_with_pre_pruning)

In [42]:
tree = Tree(width=800, height=800)
tree.add("",
         [data2],
         tree_collapse_interval=5,
         tree_top="15%",
         tree_right="20%",
         tree_symbol = 'rect',
         tree_symbol_size = 20,
         )
tree.render()
tree

In [43]:
data1_information_gain = generate_echarts_data(tree_with_pre_pruning_information_gain)
tree = Tree(width=800, height=800)
tree.add("",
         [data1_information_gain],
         tree_collapse_interval=5,
         tree_top="15%",
         tree_right="20%",
         tree_symbol = 'rect',
         tree_symbol_size = 20,
         )
tree.render()
tree

In [44]:
data2_information_gain = generate_echarts_data(tree_with_pre_pruning_information_gain)
tree = Tree(width=800, height=800)
tree.add("",
         [data2_information_gain],
         tree_collapse_interval=5,
         tree_top="15%",
         tree_right="20%",
         tree_symbol = 'rect',
         tree_symbol_size = 20,
         )
tree.render()
tree

In [45]:
data1_gini = generate_echarts_data(tree_without_pre_pruning_gini)
tree = Tree(width=800, height=800)
tree.add("",
         [data1_gini],
         tree_collapse_interval=5,
         tree_top="15%",
         tree_right="20%",
         tree_symbol = 'rect',
         tree_symbol_size = 20,
         )
tree.render()
tree

In [46]:
data2_gini = generate_echarts_data(tree_without_pre_pruning_gini)
tree = Tree(width=800, height=800)
tree.add("",
         [data2_gini],
         tree_collapse_interval=5,
         tree_top="15%",
         tree_right="20%",
         tree_symbol = 'rect',
         tree_symbol_size = 20,
         )
tree.render()
tree